<a href="https://colab.research.google.com/github/FajarKKP/AML_DeepLearning/blob/main/task_1_cap_detection/Test_YOLOv11n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount and link our gdrive to this notebook.

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.5 MB/s eta 0:00:00


In [16]:
!pip install wandb --upgrade

import os
os.environ["WANDB_MODE"] = "online"      # enable logging
os.environ["WANDB_PROJECT"] = "train"  # set project name
os.environ["WANDB_ENTITY"] = "f-kenichi-kp-none"  # optional, your W&B account



In [17]:
import wandb
wandb.login()

True

In [18]:
from ultralytics import YOLO
import yaml

# Load YAML hyp
with open("/content/drive/MyDrive/bottle_cap_project/Result/train/hyp.yaml") as f:
    hyp_dict = yaml.safe_load(f)

model = YOLO("yolo11n.pt")
model.train(
    data="/content/drive/MyDrive/bottle_cap_project/Datset/data.yaml",
    epochs=150,
    batch=8,
    imgsz=416,
    project="/content/drive/MyDrive/bottle_cap_project/Result/train",
    **hyp_dict
)


Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/bottle_cap_project/Datset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.9, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, per

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b9a6500cb30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [19]:
model = YOLO("/content/drive/MyDrive/bottle_cap_project/Result/train/train/weights/best.pt")

# Export to ONNX format
model.export(
    format="onnx",   # target format
    imgsz=320,       # input size used for training
    simplify=True    # optional: simplifies the ONNX graph for faster inference
)


Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/bottle_cap_project/Result/train/train/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 7, 2100) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 387ms
Prepared 6 packages in 2.73s
Installed 6 packages in 402ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.19.1
 + onnxruntime==1.24.0.dev20251031003
 + onnxslim==0.1.77

requirements: AutoUpdate success ✅ 4.4s
WARNING ⚠️ requirements: Restart runtime or rerun command 

'/content/drive/MyDrive/bottle_cap_project/Result/train/train/weights/best.onnx'

In [21]:
from ultralytics import YOLO

# Load your trained PyTorch model
model = YOLO("/content/drive/MyDrive/bottle_cap_project/Result/train/train/weights/best.pt")

# Validate on the test split
results = model.val(
    data="/content/drive/MyDrive/bottle_cap_project/Datset/data.yaml",
    split="test",  # use test set from data.yaml
    imgsz=320,
    batch=8,
    device="cpu"  # or "cuda"
)

# Overall mAP metrics (dataset-level)
print("\n=== Overall metrics ===")
print(f"mAP50: {results.results_dict['metrics/mAP50(B)']:.4f}")
print(f"mAP50-95: {results.results_dict['metrics/mAP50-95(B)']:.4f}")
print(f"Precision: {results.results_dict['metrics/precision(B)']:.4f}")
print(f"Recall: {results.results_dict['metrics/recall(B)']:.4f}")

# Per-class mAP50
print("\n=== Per-class mAP50 ===")
for i, name in results.names.items():
    per_class_map = results.maps[i] if hasattr(results, 'maps') else 0
    n_instances = results.nt_per_class[i] if hasattr(results, 'nt_per_class') else 0
    print(f"{name}: mAP50={per_class_map:.4f}, instances={n_instances}")

# Optional: inference speed per image
print("\n=== Speed per image (ms) ===")
speed = results.speed
print(f"Preprocess: {speed['preprocess']:.2f}")
print(f"Inference: {speed['inference']:.2f}")
print(f"Postprocess: {speed['postprocess']:.2f}")


Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.2 ms, read: 5.1±0.0 MB/s, size: 22.3 KB)
val: Scanning /content/drive/MyDrive/bottle_cap_project/Datset/test/labels.cache... 2 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2/2 1.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.8it/s 0.2s
                   all          2         14      0.858          1      0.995      0.738
                others          1          6      0.828          1      0.995       0.78
            light_blue          1          1      0.751          1      0.995      0.697
             dark_blue          1          7      0.995          1      0.995      0.736
Speed: 0.5ms preprocess, 83.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/runs/d

In [23]:
import onnxruntime as ort
import cv2
import numpy as np
import yaml
import time
import os

# Load data.yaml
with open("/content/drive/MyDrive/bottle_cap_project/Datset/data.yaml") as f:
    data_cfg = yaml.safe_load(f)

test_path = data_cfg['test']  # path to test images folder

# List all test images
image_files = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(('.jpg', '.png'))]

# Load ONNX model
onnx_model_path = "/content/drive/MyDrive/bottle_cap_project/Result/train/train/weights/best.onnx"
session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])  # or GPUExecutionProvider

img_size = 320

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    scale = img_size / max(h, w)
    nh, nw = int(h*scale), int(w*scale)
    img_resized = cv2.resize(img, (nw, nh))
    canvas = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    canvas[:nh, :nw] = img_resized
    img_norm = canvas.astype(np.float32) / 255.0
    img_chw = np.transpose(img_norm, (2,0,1))
    return np.expand_dims(img_chw, axis=0)

# Run inference and measure speed
times = []
for img_path in image_files:
    input_tensor = preprocess_image(img_path)
    start = time.time()
    outputs = session.run(None, {session.get_inputs()[0].name: input_tensor})
    end = time.time()
    times.append((end-start)*1000)  # milliseconds

if times:
    print(f"Test dataset size: {len(image_files)} images")
    print(f"Average inference time per image: {np.mean(times):.2f} ms")
    print(f"Max: {np.max(times):.2f} ms, Min: {np.min(times):.2f} ms")


Test dataset size: 2 images
Average inference time per image: 44.52 ms
Max: 44.82 ms, Min: 44.22 ms
